In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 150

In [35]:
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact

# Data
# from transat.data import HYPOTHETICAL_SUBMISSION_DATE
from transat.data.load import download_historical, load_historical
from transat.data.split import split_historical
from transat.data.transform import preprocess_historical_basic, dataframe_to_array

# Metric
from transat.metric import mae

# Scenario/Simulation
from transat.data.scenario import generate_scenario

In [42]:
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-09-30")

In [43]:
download_historical()
df = load_historical()

In [44]:
help(preprocess_historical_basic)

Help on function preprocess_historical_basic in module transat.data.transform:

preprocess_historical_basic(df, cases_col=['NewCasesSmoothed7Days'], npi_cols=['C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events', 'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements', 'C7_Restrictions on internal movement', 'C8_International travel controls', 'H1_Public information campaigns', 'H2_Testing policy', 'H3_Contact tracing', 'H6_Facial Coverings'], norm_npis=True, norm_by_pop=True)
    Create a copy of preprocessed data.



In [45]:
df = preprocess_historical_basic(df,norm_by_pop=False)

In [46]:
print("Spliting at : ", HYPOTHETICAL_SUBMISSION_DATE)
# where ``df_train`` contains all data <= ``split_date`` and ``df_test`` contains all data > ``split_date``
df_train, df_test = split_historical(df, HYPOTHETICAL_SUBMISSION_DATE)

Spliting at :  2020-09-30


In [54]:
nb_lookback_days = 1
nb_lookahead_days = 1
sequence_format = True
neg_npis = True

(X_train, y_train), (X_cols, y_col) = dataframe_to_array(df_train, nb_lookback_days=nb_lookback_days, 
    nb_lookahead_days=nb_lookahead_days, sequence_format=sequence_format, neg_npis=neg_npis)
(X_test, y_test), _ = dataframe_to_array(df_test, nb_lookback_days=nb_lookback_days,
    nb_lookahead_days=nb_lookahead_days, sequence_format=sequence_format, neg_npis=neg_npis)

X_train, y_train = X_train.reshape(X_train.shape[0], -1), y_train.reshape(-1)
X_test, y_test = X_test.reshape(X_test.shape[0], -1), y_test.reshape(-1)

print("X_train shape: ", np.shape(X_train))
print("y_train shape: ", np.shape(y_train))
print()
print("X_test  shape: ", np.shape(X_test))
print("y_test  shape: ", np.shape(y_test))

X_train shape:  (64227, 13)
y_train shape:  (64227,)

X_test  shape:  (18012, 13)
y_test  shape:  (18012,)


In [55]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.

from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=20000,
              positive=True,
              selection='random')

# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=20000, positive=True, precompute=True,
      selection='random')

In [ ]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

18013

In [79]:
from fireTS.models import NARX, DirectAutoRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np
from sklearn.linear_model import LinearRegression

In [98]:
x = np.random.randn(100, 5)
y = np.random.randn(100)
mdl = NARX(LinearRegression(), auto_order=2, exog_order=[2]*X_test.shape[1])
mdl.fit(X_train, y_train)

In [101]:
y_forecast = mdl.forecast(X_train, y_train, step=(1+X_test.shape[0]))

/home/romainegele_gmail_com/anaconda3/envs/xprize/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/romainegele_gmail_com/anaconda3/envs/xprize/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/romainegele_gmail_com/anaconda3/envs/xprize/lib/python3.7/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [90]:
print(y_forecast)

[ 0.54798463  0.17979991 -0.29581991  0.09849941 -0.32614233  0.05822209
 -0.46898912 -0.06852968 -0.07776872  0.10937544  0.14754235  0.01739174
 -0.47249891 -0.1199614   0.05125335 -0.02254128  0.36711024  0.195214
 -0.02119053  0.37865239  0.17269093]


In [76]:
# Evaluate model
train_preds = model_narx.forecast(X_train, y_train, step=1, X_future=X_test) #model_narx.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

#test_preds = model_narx.predict(X_test)
#test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
#print('Test MAE:', mae(test_preds, y_test))

ValueError: The row number of X_future (18012) must to step - 1 (0)!

Train MAE: 39.867297110558376
Test MAE: 183.49230764150812


In [57]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + X_cols[0])
for d in range(-nb_lookback_days, 1):
    for col_name in X_cols[1:]:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -1 NewCasesSmoothed7Days 1.0095020066104092
Intercept 4.677261735110051


In [58]:
def simulate_scenario(X_scenario, y_scenario):
    # Simulate scenario

    X_sim = X_scenario.copy()
    X_sim_cases = X_sim[:,:,:1]
    X_sim_npis = X_sim[:,:,1:]
    y_sim = np.zeros(np.shape(y_scenario))

    nb_lookback_days = X_sim.shape[1]

    for d in range(y_sim.shape[1]):

        y = model.predict(X_sim.reshape(1,-1))
        y_sim[0,d,0] = max(y[0], 0)

        # Assuming constant NPIs here
        X_sim_npis = np.concatenate([X_sim_npis[:,1:], X_sim_npis[:,-1:]], axis=1)
        X_sim_cases = np.concatenate([X_sim_cases[:,1:], y.reshape(-1, 1, 1)], axis=1)

        X_sim =  np.concatenate([X_sim_cases, X_sim_npis], axis=-1)
        X_sim = np.array(X_sim)
    
    return y_sim

In [59]:
def viz_scenario(geo_id, X_scenario, y_scenario, y_sim):
    mae_error = mae(y_scenario, y_sim)

    plt.figure()
    plt.title(geo_id)

    plot_input_x = np.arange(X_scenario.shape[1])
    plot_input_y = X_scenario[:,:,:1].reshape(-1)

    plt.plot(plot_input_x, plot_input_y, label="Input Scenario")

    plot_output_x = np.arange(y_scenario.shape[1])+X_scenario.shape[1]
    plot_output_x = np.concatenate([plot_input_x[-1:], plot_output_x])
    plot_output_y = np.concatenate([plot_input_y[-1:], y_scenario.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Scenario")


    plot_output_y = np.concatenate([plot_input_y[-1:], y_sim.reshape(-1)])
    plt.plot(plot_output_x, plot_output_y, label="Output Simulation")

    plt.ylabel("New Cases")
    plt.xlabel("Days")
    ax = plt.gca()
    plt.text(0.3, 0.5, f"$MAE={mae_error:.2f}$", transform=ax.transAxes)
    plt.legend()
    plt.show()

In [60]:
def interactive_scenario(geo_id):
    nb_future_days=30

    X_scenario , y_scenario = generate_scenario(df_train, df_test, geo_id, nb_lookback_days=nb_lookback_days,
        nb_future_days=nb_future_days, sequence_format=sequence_format)

    y_sim = simulate_scenario(X_scenario, y_scenario)

    viz_scenario(geo_id, X_scenario, y_scenario, y_sim)
    
geo_ids = sorted(df.GeoID.unique())

w_geo_id = widgets.Dropdown(
    options=geo_ids,
    value='France__nan',
    description='GeoID:',
    disabled=False
)

interact(interactive_scenario, geo_id=w_geo_id)

interactive(children=(Dropdown(description='GeoID:', index=60, options=('Afghanistan__nan', 'Albania__nan', 'A…

<function __main__.interactive_scenario(geo_id)>

In [61]:
from fireTS.models import NARX, DirectAutoRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

# Random training data
x = np.random.randn(100, 2)
y = np.random.randn(100,)

# Build a non-linear autoregression model with exogenous inputs
# using Random Forest regression as the base model
mdl1 = NARX(
    RandomForestRegressor(n_estimators=10),
    auto_order=2,
    exog_order=[2, 2],
    exog_delay=[1, 1])
mdl1.fit(x, y)
ypred1 = mdl1.predict(x, y, step=3)

# Build a general autoregression model and make multi-step prediction directly
# using XGBRegressor as the base model
# mdl2 = DirectAutoRegressor(
#     XGBRegressor(n_estimators=10),
#     auto_order=2,
#     exog_order=[2, 2],
#     exog_delay=[1, 1],
#     pred_step=3)
# mdl2.fit(x, y)
# ypred2 = mdl2.predict(x, y)

/home/romainegele_gmail_com/anaconda3/envs/xprize/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.regression module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [62]:
np.shape(ypred2)

NameError: name 'ypred2' is not defined

In [63]:
np.shape(ypred1)

(100,)

In [64]:
mdl1.forecast(x,y)

ValueError: Number of features of the model must match the input. Model n_features is 6 and input n_features is 4 